In [1]:
import numpy as np
import cv2

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def extract_video_first_frame(path, use_cv2=True):
    
    cap = cv2.VideoCapture(path)
    
    ret, frame = cap.read()
    
    if use_cv2:
        cv2.imshow('First Frame', frame)

        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    else:
        new_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        plt.imshow(new_frame)
        plt.axis('off')
        
    return frame

In [3]:
# PATH = './data/videos/rcpc_bimbambum.mp4'
# img = extract_video_first_frame(PATH)

In [23]:
proto_file = './openpose/openpose-master/models/pose/coco/pose_deploy_linevec.prototxt'
weight_file = './openpose/openpose-master/models/pose/coco/pose_iter_440000.caffemodel'

img = cv2.imread('./data/basketball.jpg')
img_copy = img.copy()

In [24]:
frame_width, frame_height = img.shape[1], img.shape[0]
in_height = 368
in_width = 368

pose_net = cv2.dnn.readNetFromCaffe(proto_file, weight_file)
pose_blob = cv2.dnn.blobFromImage(img_copy, scalefactor=1.0/255, size=(in_width, in_height), mean=(0,0,0), swapRB=False, crop=False)
pose_net.setInput(pose_blob)
output = pose_net.forward()

In [ ]:
len(output)

In [ ]:
i = 0

prob_map = output[0,i,:,:]
prob_map = cv2.resize(prob_map, (frame_width, frame_height))

plt.imshow(cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB))    
plt.imshow(prob_map, alpha=0.5)
plt.show()

In [ ]:
threshold = 0.1

map_gauss = cv2.GaussianBlur(prob_map, (3,3), 0, 0)
map_mask = np.uint8(map_gauss > threshold)

In [ ]:
contours, _ = cv2.findContours(map_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
body_points = []

for c in contours:
    blob_mask = np.zeros(map_mask.shape)
    blob_mask = cv2.fillConvexPoly(blob_mask, c, 1)
    masked_prob_map = map_gauss * blob_mask
    _, max_val, _, max_loc = cv2.minMaxLoc(masked_prob_map)
    
    body_points.append(max_loc + (prob_map[max_loc[1], max_loc[0]],))

In [ ]:
for i in range(len(body_points)):
    cv2.circle(img_copy, (body_points[i][0], body_points[i][1]), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)

cv2.imshow('Body Points', img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [33]:
img_copy = img.copy()

n_points = 18 # COCO Pose
threshold = 0.1
body_points = []

for i in range(n_points):
    prob_map = output[0,i,:,:]
    prob_map = cv2.resize(prob_map, (frame_width, frame_height))
    
    map_gauss = cv2.GaussianBlur(prob_map, (3,3), 0, 0)
    map_mask = np.uint8(map_gauss > threshold)
    
    contours, _ = cv2.findContours(map_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for c in contours:
        blob_mask = np.zeros(map_mask.shape)
        blob_mask = cv2.fillConvexPoly(blob_mask, c, 1)
        masked_prob_map = map_gauss * blob_mask
        _, max_val, _, max_loc = cv2.minMaxLoc(masked_prob_map)

        body_points.append(max_loc + (prob_map[max_loc[1], max_loc[0]],))
        
for i in range(len(body_points)):
    cv2.circle(img_copy, (body_points[i][0], body_points[i][1]), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)

cv2.imshow('Body Points', img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [36]:
proto_file = './openpose/openpose-master/models/pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt'
weight_file = './openpose/openpose-master/models/pose/mpi/pose_iter_160000.caffemodel'

frame_width, frame_height = img.shape[1], img.shape[0]
in_height = 368
in_width = 368

pose_net = cv2.dnn.readNetFromCaffe(proto_file, weight_file)
pose_blob = cv2.dnn.blobFromImage(img_copy, scalefactor=1.0/255, size=(in_width, in_height), mean=(0,0,0), swapRB=False, crop=False)
pose_net.setInput(pose_blob)
output = pose_net.forward()

img = cv2.imread('./data/basketball.jpg')
img_copy = img.copy()

h, w = output.shape[2], output.shape[3]
threshold = 0.1
n_points = 44 # MPI Pose

body_points = []
for i in range(n_points):
    probs = output[0,i,:,:]
    min_val, prob, min_loc, point = cv2.minMaxLoc(probs)
    
    if prob > threshold:
        
        x = (frame_width * point[0]) / w
        y = (frame_height * point[1]) / h
        
        cv2.circle(img_copy, (int(x), int(y)), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
        cv2.putText(img_copy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 3, lineType=cv2.LINE_AA)
        
        body_points.append((int(x), int(y)))
        
    else:
        body_points.append(None)
        
cv2.imshow('Body Points', img_copy)    
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
body_points